In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import json

In [245]:


element = soup1.find_all("script")

#print(element[19])
review = element[16].string

#delete _data part and the extra ; at the end
review = review[28: len(review)-1]

In [246]:
data = json.loads(review)

In [249]:
ls2=[]
for k in range(len(data['listing']['reviews']['collection'])):
    z2=data['listing']['reviews']['collection'][k]['rating']
    ls2.append(z2)

In [299]:
############ Galaxy #############

reviews=[]
ratings=[]

page=6
for p in range(1, page+1):
    url = "https://www.productreview.com.au/listings/samsung-galaxy-s9?page="+str(p)+"#reviews"
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    zz=soup.find_all(itemprop="description")
    
    for i in range(len(zz)):
        j=zz[i].getText()
        reviews.append(j)
    
    element = soup.find_all("script")
    review = element[16].string
    review = review[28: len(review)-1]
    data = json.loads(review)
    
    for k in range(len(data['listing']['reviews']['collection'])):
        zzz=data['listing']['reviews']['collection'][k]['rating']
        ratings.append(zzz)

In [310]:
############ iPhone #############
reviews1=[]
ratings1=[]

page=33
for p in range(1, page+1):
    url = "https://www.productreview.com.au/listings/apple-iphone-x?page="+str(p)+"#reviews"
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    zz=soup.find_all(itemprop="description")
    
    for i in range(len(zz)):
        j=zz[i].getText()
        reviews1.append(j)
    
    element = soup.find_all("script")
    review = element[16].string
    review = review[28: len(review)-1]
    data = json.loads(review)
    
    for k in range(len(data['listing']['reviews']['collection'])):
        zzz=data['listing']['reviews']['collection'][k]['rating']
        ratings1.append(zzz)

In [311]:
len(reviews1)
len(ratings1)

660

In [324]:
import numpy as np
import pandas as pd

phone = list(np.repeat("galaxy",len(ratings)))
source = list(np.repeat("Productreview.com.au",len(ratings))) 

phone1 = list(np.repeat("iphone",len(ratings1)))
source1 = list(np.repeat("Productreview.com.au",len(ratings1)))


In [327]:
galaxy = pd.DataFrame({"source":source, "phone":phone, "review": reviews, "rating":ratings})
iPhone = pd.DataFrame({"source":source1, "phone":phone1, "review": reviews1, "rating":ratings1})

In [334]:
#conbining rows
productreview=pd.concat([galaxy, iPhone], axis=0)

In [336]:
bestbuy = pd.read_csv("bestbuy.csv")

In [338]:
df = pd.concat([productreview, bestbuy], axis=0)

In [340]:
df.to_csv('sofar.csv')

In [1]:
##### Loading
import numpy as np
import pandas as pd

In [37]:
df = pd.read_csv('sofar.csv')

In [25]:
df1=pd.read_csv('influenster_galaxy.csv')
df2=pd.read_csv('influenster_iphone.csv')

In [51]:
data = pd.concat([df, df1, df2], axis=0)

In [55]:
#df=df.drop("Unnamed: 0", axis=1)
#data.to_csv("data.csv", index=False)

In [56]:
data = pd.read_csv("data.csv")

In [63]:
#Cleaning

data["phone"].value_counts()

for i in range(len(data["phone"])):
    if data["phone"][i] == "iPhone":
        data["phone"][i] = "iphone"
    

C:\Users\tkjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [71]:
data["phone"].value_counts()

galaxy    1812
iphone    1689
Name: phone, dtype: int64

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
from bs4 import BeautifulSoup

In [290]:
driver = webdriver.Chrome()
driver.get("https://www.very.co.uk/samsung-galaxynbsps9-64gb-blue/1600255626.prd?_requestid=1300638&Ntt=s9#BVRRContainer")

# need to navigate through reviews.
# problem for this website is that the url doesn't change no matter what I navigate through
# but selenium can do it

#this is to get to the reviews from the original url
#review_button_path = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/a'
#wait = WebDriverWait(driver, 5)
#review_button = wait.until(EC.visibility_of_element_located((By.XPATH, review_button_path)))
#driver.execute_script("arguments[0].click();", review_button)

In [300]:
############################ Galaxy #############################
#################################################################

#################################################################
'''
WARNING:

This code chunks take about 50 minutes to run
'''
#################################################################

# this below retreives all the text in the review container
# but don't need it....
#review_container = driver.find_element_by_id("BVRRContainer")

# get the reviews and rating for the first page
reviews = []
ratings = []
review_rating = []

review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
for el in review_list:
    #print(el.find_element_by_class_name("bv-content-summary-body-text").text)
    #print(el.find_element_by_class_name("bv-rating-ratio").text)
    reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
    ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)

In [305]:
from selenium.common import exceptions
import time

# get the reviews starting from the 2nd page
# until there is no button to click
# there are about 37ish pages total since there are about 1100 reviews
# (probably) 30 pages per reviews except for the first page
i=2
reviews_ratings=[]
#wait = WebDriverWait(driver, 30)

while(True):
    try :
        next_review_path = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/a' 
        review_next_button = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(3)
  
        # append the reviews and ratings to a list
        review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
        for el in review_list:
            reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
            ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)
        time.sleep(3)
        print(i)
        i=i+1
    
    #except exceptions.StaleElementReferenceException:
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


In [349]:
#verydata = pd.DataFrame({"review":reviews, "rating":ratings})
#verydata.to_csv("very_galaxy.csv", index=False)

In [320]:
############################ iPhone 1 #############################
#################################################################

driver = webdriver.Chrome()
driver.get("https://www.very.co.uk/apple-iphonenbspxnbsp64gbnbsp--space-grey/1600213805.prd#BVRRContainer")

In [321]:
reviews = []
ratings = []
review_rating = []

review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
for el in review_list:
    #print(el.find_element_by_class_name("bv-content-summary-body-text").text)
    #print(el.find_element_by_class_name("bv-rating-ratio").text)
    reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
    ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)

In [322]:
from selenium.common import exceptions
import time

# get the reviews starting from the 2nd page
# until there is no button to click
# there are about 37ish pages total since there are about 1100 reviews
# (probably) 30 pages per reviews except for the first page
i=2
reviews_ratings=[]
#wait = WebDriverWait(driver, 30)

while(True):
    try :
        next_review_path = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/a' 
        review_next_button = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(3)
  
        # append the reviews and ratings to a list
        review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
        for el in review_list:
            reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
            ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)
        time.sleep(3)
        print(i)
        i=i+1
    
    #except exceptions.StaleElementReferenceException:
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

2
3
4
5
6
7


In [324]:
verydata_a1 = pd.DataFrame({"reviews":reviews, "ratings":ratings})
#verydata_a1.to_csv("verydata_a1.csv", index=False)

In [328]:
############################ iPhone 2 #############################
#################################################################

driver = webdriver.Chrome()
driver.get("https://www.very.co.uk/apple-iphonenbspx-64gb--nbspsilver/1600213795.prd#BVRRContainer")

reviews = []
ratings = []
review_rating = []

review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
for el in review_list:
    #print(el.find_element_by_class_name("bv-content-summary-body-text").text)
    #print(el.find_element_by_class_name("bv-rating-ratio").text)
    reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
    ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)

    

from selenium.common import exceptions
import time

# get the reviews starting from the 2nd page
# until there is no button to click
# there are about 37ish pages total since there are about 1100 reviews
# (probably) 30 pages per reviews except for the first page
i=2
reviews_ratings=[]
#wait = WebDriverWait(driver, 30)

while(True):
    try :
        next_review_path = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/a' 
        review_next_button = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(3)
  
        # append the reviews and ratings to a list
        review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
        for el in review_list:
            reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
            ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)
        time.sleep(3)
        print(i)
        i=i+1
    
    #except exceptions.StaleElementReferenceException:
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

2
3
4
5
6
7


In [329]:
verydata_a2 = pd.DataFrame({"reviews":reviews, "ratings":ratings})

In [331]:
############################ iPhone 3 #############################
#################################################################

driver = webdriver.Chrome()
driver.get("https://www.very.co.uk/apple-iphone-xnbsp256gbnbsp--space-grey/1600213807.prd#BVRRContainer")

reviews = []
ratings = []
review_rating = []

review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
for el in review_list:
    #print(el.find_element_by_class_name("bv-content-summary-body-text").text)
    #print(el.find_element_by_class_name("bv-rating-ratio").text)
    reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
    ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)

    

from selenium.common import exceptions
import time

# get the reviews starting from the 2nd page
# until there is no button to click
# there are about 37ish pages total since there are about 1100 reviews
# (probably) 30 pages per reviews except for the first page
i=2
reviews_ratings=[]
#wait = WebDriverWait(driver, 30)

while(True):
    try :
        next_review_path = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/a' 
        review_next_button = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(3)
  
        # append the reviews and ratings to a list
        review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
        for el in review_list:
            reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
            ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)
        time.sleep(3)
        print(i)
        i=i+1
    
    #except exceptions.StaleElementReferenceException:
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

2
3
4


In [332]:
verydata_a3 = pd.DataFrame({"reviews":reviews, "ratings":ratings})

In [333]:
len(verydata_a3)

89

In [334]:
############################ iPhone 4 #############################
#################################################################

driver = webdriver.Chrome()
driver.get("https://www.very.co.uk/apple-iphone-xnbsp256gbnbsp--silver/1600213784.prd#BVRRContainer")

reviews = []
ratings = []
review_rating = []

review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
for el in review_list:
    #print(el.find_element_by_class_name("bv-content-summary-body-text").text)
    #print(el.find_element_by_class_name("bv-rating-ratio").text)
    reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
    ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)

    

from selenium.common import exceptions
import time

# get the reviews starting from the 2nd page
# until there is no button to click
# there are about 37ish pages total since there are about 1100 reviews
# (probably) 30 pages per reviews except for the first page
i=2
reviews_ratings=[]
#wait = WebDriverWait(driver, 30)

while(True):
    try :
        next_review_path = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/a' 
        review_next_button = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(3)
  
        # append the reviews and ratings to a list
        review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
        for el in review_list:
            reviews.append(el.find_element_by_class_name("bv-content-details-offset-on").text)
            ratings.append(el.find_element_by_class_name("bv-rating-stars-container").find_element_by_class_name("bv-off-screen").text)
        time.sleep(3)
        print(i)
        i=i+1
    
    #except exceptions.StaleElementReferenceException:
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

2
3


In [335]:
verydata_a4 = pd.DataFrame({"reviews":reviews, "ratings":ratings})

In [336]:
len(verydata_a4)

55

In [337]:
very_apple = pd.concat([verydata_a1, verydata_a2, verydata_a3, verydata_a4], axis=0)

In [339]:
very_apple.to_csv("very_iphone.csv", index=False)

In [350]:
########################## Cleaning ##########################

#very_iphone = pd.read_csv("very_iphone.csv")
very_galaxy = pd.read_csv("very_galaxy.csv")

In [346]:
import numpy as np

phone_g = list(np.repeat("galaxy",len(very_galaxy)))
source_g = list(np.repeat("Very.co.uk",len(very_galaxy))) 
list_g = pd.DataFrame({"phone":phone_g, "source":source_g})


phone_a = list(np.repeat("iphone",len(very_iphone)))
source_a = list(np.repeat("Very.co.uk",len(very_iphone)))
list_a = pd.DataFrame({"phone":phone_a, "source":source_a})

In [359]:
#very_galaxy = pd.read_csv("verydata.csv")
#very_galaxy.to_csv("very_galaxy.csv", index=False)

In [382]:
very_galaxy1 = pd.concat([very_galaxy, list_g], axis = 1)
#very_galaxy1.to_csv("very_galaxy.csv", index = False)
very_galaxy = pd.read_csv("very_galaxy.csv")
len(very_galaxy)

7620

In [376]:
very_iphone = pd.read_csv("very_iphone.csv")
very_iphone1 = pd.concat([very_iphone, list_a], axis = 1)


In [379]:
very_iphone1.to_csv("very_iphone.csv", index=False)

In [384]:
#very_iphone = pd.read_csv("very_iphone.csv")
verydata = pd.concat([very_iphone, very_galaxy], axis = 0)
#verydata.to_csv("verydata.csv", index=False)

In [437]:
verydata = pd.read_csv("verydata.csv")

In [409]:
######## Cleaning reviews ########


# "\r\r\r\n" is a sufficient for splitting

review_list=[]

for i in verydata.review:
    s=i.split("\r\r\r\n")[0]
    ss=s.strip()
    review_list.append(ss)

In [423]:
#verydata.drop('review', axis=1)
review = pd.DataFrame({"review":review_list})

In [465]:
######## Cleaning ratings ########
rating_list=[]
for r in verydata.rating:    
    rr=r[0]
    rating_list.append(int(rr))
    

In [434]:
rating = pd.DataFrame({"rating":rating_list})

In [456]:
verydata1 = verydata.drop(['review', 'rating'], axis=1)

verydata_final = pd.concat([verydata1, review, rating], axis=1)

In [485]:
################## Fianl Combining ####################
#######################################################

sofar = pd.read_csv("data/data.csv")
verydata_final = pd.read_csv("data/verydata_final.csv")
vw_galaxy = pd.read_csv("data/vw_galaxy.csv")
vw_iphone = pd.read_csv("data/vw_iphone.csv")
att_galaxy = pd.read_csv("data/att_galaxy.csv")
att_iphone = pd.read_csv("data/att_iphone.csv")

In [487]:
vw_rating=[]
for k in vw_iphone.rating:
    kk=k[0]
    vw_rating.append(int(kk))

rating = pd.DataFrame({"rating":vw_rating})
vw_iphone = vw_iphone.drop(['rating'], axis=1)
vw_iphone_m = pd.concat([vw_iphone, rating], axis=1)

In [489]:
vw_iphone_m.to_csv("data/vw_iphone_m.csv", index=False)
vw_iphone = pd.read_csv("data/vw_iphone_m.csv")

In [490]:
final_data = pd.concat([sofar, verydata_final, vw_galaxy, vw_iphone, att_galaxy, att_iphone], axis=0)

In [491]:
final_data.to_csv("data/final_data.csv", index=False)

In [493]:
final_data['phone'].value_counts()

galaxy    9775
iphone    3476
Name: phone, dtype: int64